In [2]:
#import dataset class
from dataset import energyProject_dataset
from torch.utils.data import DataLoader
import pickle
import torch
import torch.nn as nn
import numpy as np
import tqdm
import torch
import torch.nn as nn
import pandas as pd
import yaml

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import wandb

!wandb login

wandb: Currently logged in as: haradai (energy_project_uab). Use `wandb login --relogin` to force relogin


In [4]:
#load dataset object file
with (open('data/dataset_class.pkl', "rb")) as openfile:
    dataset = pickle.load(openfile)

In [5]:
#now you can create a dataloader and use it!
dataloader = DataLoader(dataset, batch_size=100, shuffle=False)

In [17]:
dataset.__getitem__?

Signature: dataset.__getitem__(index)
Docstring:
This function will return more than one object depending on the mode it is on.
Activity encoding mode:
    0: I.very large one-hot encoding of all the combinations of classroom and activity concatenated
        with all other features.
       II. target values
    
    1:  I. 0 but first with some PCA applied to reduce dimensionality of 
        the enormous one-hot encoding.
        II. target values
    
    2: returns four objects, 
        I.mean_encoding of activity from bert(as many as activities at time stamp), 
        II. classroom one hot encoding for each activity(as many as activities at time stamp)
        III. All other features at that time stamp
        IV. target values
    
    2.5:  returns four objects
        I. embedding tensor of activity from bert(as many as activities at time stamp), 
        II. classroom one hot encoding for each activity(as many as activities at time stamp)
        III. All other features at t

In [19]:
dataset.activitivity_encoding_mode = 2 #or any value

#batch of 100:

In [18]:
next(iter(dataloader))["ocu_ber_emb"].shape

KeyError: 'ocu_ber_emb'

In [44]:
next(iter(dataloader))["espai_enc"].shape

tensor([[53., 53., 53.,  ..., 53., 53., 53.],
        [53., 53., 53.,  ..., 53., 53., 53.],
        [53., 53., 53.,  ..., 53., 53., 53.],
        ...,
        [53., 53., 53.,  ..., 53., 53., 53.],
        [53., 53., 53.,  ..., 53., 53., 53.],
        [53., 53., 53.,  ..., 53., 53., 53.]])

In [43]:
next(iter(dataloader))["general_data"].shape

torch.Size([100, 21])

In [23]:
len(dataset.espais_vocab)

54

In [20]:
next(iter(dataloader))["y"].shape

torch.Size([100, 3])

In [26]:
from icecream import ic

In [6]:
BERT_DIM_EMB = 768
TIME_CLIMATE_DIM = 21
#classe = activitat + espai encoded

class attentive_model_pepemarti(nn.Module):
    def __init__(self, espai_emb_dim,hidden_dim,lstm_nl,target_dim):
        super(attentive_model_pepemarti, self).__init__()
        self.espai_emb_dim = espai_emb_dim
        self.hidden_dim = hidden_dim
        self.lstm_nl = lstm_nl

        self.espai_emb = nn.Embedding(num_embeddings=len(dataset.espais_vocab),embedding_dim=espai_emb_dim)
        self.classe_repr = nn.Sequential(
            nn.Linear(in_features=BERT_DIM_EMB+espai_emb_dim,out_features=601),
            nn.ReLU(),
            nn.Linear(in_features=601,out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256,out_features=51),
            nn.ReLU(),
            nn.Linear(in_features=51,out_features=TIME_CLIMATE_DIM),
            nn.Softmax(dim=-1)
        )

        self.key_gen =  nn.Sequential(
            nn.Linear(in_features=TIME_CLIMATE_DIM,out_features=40),
            nn.ReLU(),
            nn.Linear(in_features=40,out_features=hidden_dim),
            nn.ReLU(),
        )

        self.val_gen =  nn.Sequential(
            nn.Linear(in_features=TIME_CLIMATE_DIM,out_features=40),
            nn.ReLU(),
            nn.Linear(in_features=40,out_features=hidden_dim),
            nn.ReLU(),
        )

        self.query_gen =  nn.Sequential(
            nn.Linear(in_features=TIME_CLIMATE_DIM,out_features=40),
            nn.ReLU(),
            nn.Linear(in_features=40,out_features=hidden_dim),
            nn.ReLU(),
        )
        
        self.multihead_attn = nn.MultiheadAttention(hidden_dim,3,batch_first=True)
        
        self.lstm = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=self.lstm_nl, dropout=0, batch_first=True) #input -> BATCH, SEQ_LENGHT, EMBEDDING_DIM
        
        self.regressFC = nn.Sequential(
            nn.Linear(in_features=self.hidden_dim,out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64,out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256,out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64,out_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16,out_features=target_dim),
        )

    def forward(self,ocu_ber_emb,espai_enc,general_data,h,c):
        #embedd the espais
        espai_emb = self.espai_emb(espai_enc.to(torch.int)).float()
        #concatenate activity and espai vector
        act_cat_espai = torch.cat((ocu_ber_emb,espai_emb),axis=2)

        #pass act_cat_espai through a FC to have a "classe" representation
        classe_repr = self.classe_repr(act_cat_espai.float())
    
        #get this representation and multiply it with the general data "sort of attention here"
        #first extend the general data so we have a pair for each class vector
        general_rep = general_data.unsqueeze(1).repeat(1, 34, 1)
        general_at_per_class = general_rep * classe_repr #element wise multiplication
        general_at_per_class = general_at_per_class.float() #convert to float32

        key = self.key_gen(general_at_per_class) #generate key with FC
        value = self.val_gen(general_at_per_class) #generate value with FC
        query = self.query_gen(general_at_per_class) #generate query with FC
        
        att_mask = torch.sum(ocu_ber_emb,axis=2) != 0.0 #look for what values have to enter. The ones that aren't padding
        att_mask = att_mask.unsqueeze(1)
        att_mask = att_mask.repeat(3, att_mask.shape[2], 1)#repeat 3 times along batch dim because we have three heads. The other repeat is because we need it to be of shape seqlenxseqlen 
        output, _ =  self.multihead_attn(key=key, value=value, query=query,attn_mask=att_mask)
        output = torch.sum(output,axis=1)

        #create extra to output be a sequence of 1
        output = output.unsqueeze(1)
        
        #pass this as the inital state to a LSTM
        out, (h,c) = self.lstm(output, (h,c))
        out =  self.regressFC(out).float()
        return out, h, c
    
    def init_hidden(self,batch_size):
        # Initialize the hidden state and cell state with zeros
        h = torch.zeros(self.lstm_nl, batch_size, self.hidden_dim,dtype=torch.float32)
        c = torch.zeros(self.lstm_nl, batch_size, self.hidden_dim,dtype=torch.float32)
        return h, c
    

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

NameError: name 'model' is not defined

In [531]:
model = attentive_model_pepemarti(espai_emb_dim=30,hidden_dim=30,lstm_nl=1,target_dim=1)

In [532]:
h, c = model.init_hidden(batch_size)
c.shape
#model.forward(sample["ocu_ber_emb"],sample["espai_enc"],sample["general_data"],h,c)

torch.Size([1, 100, 30])

Define train function:

In [11]:
device = "mps"

In [12]:
model = attentive_model_pepemarti(espai_emb_dim=30,hidden_dim=30,lstm_nl=1,target_dim=1)

In [13]:
model = model.to(device)

In [14]:
#now you can create a dataloader and use it!
batch_size = 100
dataloader = DataLoader(dataset, batch_size, shuffle=False)

In [15]:
import torch.optim as optim
def train(dataloader, model, batch_size, num_epochs):
    model.train()

    criterion =  nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    losses = []
    for epoch in range(num_epochs):
        for batch, data in enumerate(dataloader):
            ocu_emb, espai_enc, general_data = data["ocu_ber_emb"].float().to(device) ,data["espai_enc"].float().to(device) ,data["general_data"].float().to(device)
            y = data["y"][:,0].float().to(device) #we'll do one counter for now
            optimizer.zero_grad()

            #current batch size size
            b_sz = ocu_emb.shape[0]

            #note the dataloader with a batch of 100 when reachs the end expects a batch of 60
            h, c = model.init_hidden(b_sz) # Start with a new state in each batch            
            h = h.to(device)
            c = c.to(device)

            try:
                y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
            except:
                print("error!")
                ic(batch)
                ic(ocu_emb.shape)
                ic(espai_enc.shape)
                ic(general_data.shape)
                ic(h.shape)
                ic(c.shape)
                y_pred, h,c= model(ocu_emb, espai_enc, general_data, h, c)
            y_pred = y_pred[:,0,0]
            loss = criterion(y_pred,y)  #cross entropy loss needs (N,C,seq_lenght)

            loss.backward()
            optimizer.step()

            if batch%30 == 0:
                print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
                losses.append(loss.item())
    return losses

In [20]:
num_epochs = 20
losses = train(dataloader, model, batch_size=100, num_epochs=num_epochs)

{'epoch': 0, 'batch': 0, 'loss': 886.621826171875}
{'epoch': 0, 'batch': 30, 'loss': 6231.90380859375}
{'epoch': 0, 'batch': 60, 'loss': 6700.3837890625}


KeyboardInterrupt: 